<a href="https://colab.research.google.com/github/Schrodingerscat00000/Hand-Raise-Detection-from-Video/blob/main/Hand_Raise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
pip install ultralytics opencv-python mediapipe numpy --quiet


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.


In [ ]:
!pip install deepface


In [ ]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.9/292.9 kB 25.5 MB/s eta 0:00:00


In [ ]:
import cv2
import pickle
import numpy as np
from deepface import DeepFace

SETUP_IMG_PATH = "/content/drive/MyDrive/fringe core/IMG.png"
DB_PATH = "faces.db"
NAMES = ["TANVIR", "SHAFAYET", "TOUFIQ", "FAISAL", "MUFRAD", "ANIK", "IMRAN", "EMON"]
DETECTOR_BACKEND = "opencv"

def crop_face(img, area):
    x, y, w, h = area["x"], area["y"], area["w"], area["h"]
    return img[y:y+h, x:x+w]

img_bgr = cv2.imread(SETUP_IMG_PATH)
if img_bgr is None:
    raise FileNotFoundError(f"Could not load image at {SETUP_IMG_PATH}")
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

extracted = DeepFace.extract_faces(
    img_path=img_rgb,
    detector_backend=DETECTOR_BACKEND,
    enforce_detection=True
)

if not extracted:
    raise RuntimeError("No faces found in setup image!")

if len(extracted) != len(NAMES):
    print(f"[WARN] Found {len(extracted)} faces, but have {len(NAMES)} names.")

for e in extracted:
    area = e["facial_area"]
    x, y, w, h = area["x"], area["y"], area["w"], area["h"]
    e["cx"] = x + w // 2
    e["cy"] = y + h // 2

extracted.sort(key=lambda e: e["cy"])
mid = len(extracted) // 2
top_row = sorted(extracted[:mid], key=lambda e: e["cx"])
bottom_row = sorted(extracted[mid:], key=lambda e: e["cx"])
ordered = top_row + bottom_row

face_db = {}
print("Generating embeddings…")
for name, face_info in zip(NAMES, ordered):
    crop_bgr = crop_face(img_bgr, face_info["facial_area"])
    try:
        rep = DeepFace.represent(
            img_path=crop_bgr,
            model_name="VGG-Face",
            enforce_detection=True
        )[0]["embedding"]
        face_db[name] = rep
        print(f"{name}")
    except Exception as e:
        print(f"[WARN] {name} embedding failed: {e}")

with open(DB_PATH, "wb") as f:
    pickle.dump(face_db, f)
print(f"Saved {len(face_db)} embeddings to '{DB_PATH}'")


[WARN] Found 1 faces, but have 8 names.
Generating embeddings…
TANVIR
Saved 1 embeddings to 'faces.db'


In [ ]:
import cv2
import pickle
import collections
import numpy as np
from scipy.spatial.distance import cdist
from ultralytics import YOLO
from deepface import DeepFace

VIDEO_IN = "/content/drive/MyDrive/fringe core/desk_video.mp4"
VIDEO_OUT = "hand_raise_output.mp4"
SIM_THR = 0.55
HIST_F = 7
HOLD_FRAMES = 5
FONT = cv2.FONT_HERSHEY_SIMPLEX
MAX_DIST = 100

class CentroidTracker:
    def __init__(self):
        self.next_id = 0
        self.centroids = {}
        self.lost_frames = {}

    def update(self, detections):
        if not self.centroids:
            for c in detections:
                self.centroids[self.next_id] = c
                self.lost_frames[self.next_id] = 0
                self.next_id += 1
        else:
            existing_ids = list(self.centroids.keys())
            existing = np.array([self.centroids[i] for i in existing_ids])
            new = np.array(detections) if detections else np.empty((0,2))
            D = cdist(existing, new) if new.size else np.empty((existing.shape[0], 0))

            assigned_rows, assigned_cols = set(), set()
            if new.size:
                rows, cols = np.unravel_index(np.argsort(D, axis=None), D.shape)
                for r, c in zip(rows, cols):
                    if r in assigned_rows or c in assigned_cols:
                        continue
                    if D[r, c] > MAX_DIST:
                        continue
                    tid = existing_ids[r]
                    self.centroids[tid] = tuple(new[c])
                    self.lost_frames[tid] = 0
                    assigned_rows.add(r)
                    assigned_cols.add(c)

            for i, center in enumerate(detections):
                if i not in assigned_cols:
                    self.centroids[self.next_id] = center
                    self.lost_frames[self.next_id] = 0
                    self.next_id += 1

            for i, tid in enumerate(existing_ids):
                if i not in assigned_rows:
                    self.lost_frames[tid] += 1

            to_delete = [tid for tid, lost in self.lost_frames.items() if lost > HIST_F]
            for tid in to_delete:
                del self.centroids[tid]
                del self.lost_frames[tid]

        return [(tid, *self.centroids[tid]) for tid in self.centroids]

pose_model = YOLO("yolov8n-pose.pt")
tracker = CentroidTracker()
FACES_DB = pickle.load(open("faces.db", "rb"))
names = list(FACES_DB.keys())
embs = np.stack([FACES_DB[n] for n in names])

history = collections.defaultdict(lambda: collections.deque(maxlen=HIST_F))
cooldown = collections.defaultdict(int)
id2name = {}

cap = cv2.VideoCapture(VIDEO_IN)
if not cap.isOpened():
    raise RuntimeError(f"Could not open video: {VIDEO_IN}")

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 25
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(VIDEO_OUT, fourcc, fps, (w, h))

def recognize(crop):
    rep = DeepFace.represent(
        img_path=crop,
        model_name="VGG-Face",
        enforce_detection=False
    )[0]["embedding"]
    sims = 1 - np.array([np.dot(rep, e) / (np.linalg.norm(rep) * np.linalg.norm(e)) for e in embs])
    best = np.argmax(sims)
    return names[best] if sims[best] > SIM_THR else "UNKNOWN"

def is_hand_up(kp, box):
    x1, y1, x2, y2 = box
    H = y2 - y1
    def ny(i): return (kp[i][1] - y1) / float(H)
    nose = ny(0)
    ls, rs = ny(5), ny(6)
    le, re = ny(7), ny(8)
    lw, rw = ny(9), ny(10)
    left_up = lw < ls and le < ls and lw < nose
    right_up = rw < rs and re < rs and rw < nose
    return left_up or right_up

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = pose_model(frame, verbose=False)[0]
    poses = []
    for xyxy, kps in zip(results.boxes.xyxy.cpu().numpy(), results.keypoints.xy.cpu().numpy()):
        x1, y1, x2, y2 = map(int, xyxy)
        poses.append(((x1, y1, x2, y2), kps))

    centers = [((x1 + x2) / 2, (y1 + y2) / 2) for (x1, y1, x2, y2), _ in poses]
    tracks = tracker.update(centers)

    raised = set()
    for tid, cx, cy in tracks:
        dists = [np.hypot(cx - ((x1+x2)/2), cy - ((y1+y2)/2)) for (x1, y1, x2, y2), _ in poses]
        if not dists:
            continue
        idx = int(np.argmin(dists))
        x1, y1, x2, y2 = poses[idx][0]
        kp = poses[idx][1]

        if tid not in id2name:
            y0 = max(0, y1 - 40)
            y1b = min(h, y1 + 80)
            head_crop = frame[y0:y1b, x1:x2]
            id2name[tid] = recognize(head_crop)
        name = id2name[tid]

        up = is_hand_up(kp, (x1, y1, x2, y2))
        if up:
            history[tid].append(True)
            cooldown[tid] = HOLD_FRAMES
        elif cooldown[tid] > 0:
            history[tid].append(True)
            cooldown[tid] -= 1
        else:
            history[tid].append(False)

        if sum(history[tid]) > 0.6 * len(history[tid]):
            raised.add(name)

        color = (0, 0, 255) if name in raised else (0, 255, 0)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, name, (x1, y1 - 10), FONT, 0.7, color, 2, cv2.LINE_AA)

    banner = "Raised: " + (", ".join(sorted(raised)) or "None")
    cv2.rectangle(frame, (0, 0), (w, 30), (0, 0, 0), -1)
    cv2.putText(frame, banner, (10, 20), FONT, 0.6, (255, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()
print(f"Processing complete — output saved to {VIDEO_OUT}")


✅ Processing complete — output saved to hand_raise_output.mp4
